<a href="https://colab.research.google.com/github/PavankumarUppar/SMA/blob/main/SMAProgram3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install node2vec

import networkx as nx

from node2vec import Node2Vec


# Step 1: Prepare input data

# Create a graph using NetworkX

G = nx.Graph()

# Add nodes to the graph

G.add_nodes_from([1, 2, 3, 4, 5])

# Add edges to the graph

G.add_edges_from([(1, 2), (1, 3), (2, 3), (2, 4), (3, 4), (4, 5)])


# Step 2: Node2Vec algorithm

# Create a Node2Vec object with desired parameters

node2vec = Node2Vec(G, dimensions=128, walk_length=30, num_walks=200, p=1.0, q=1.0, workers=4)

# Generate walks

walks = node2vec.walks


# Step 3: Graph Embedding

# Train the Node2Vec model

model = node2vec.fit(window=10, min_count=1)


# Step 4: Link Prediction

# Predict links by comparing embeddings of node pairs

node1 = 1

node2 = 5

similarity_score = model.wv.similarity(str(node1), str(node2))


# Step 5: Recommendation Engine

# Recommend top-n links based on similarity scores

n = 3

similar_nodes = model.wv.most_similar(str(node1), topn=n)


# Print similarity score and recommendations

print(f"Similarity score between node {node1} and node {node2}: {similarity_score}")

print(f"Top {n} recommendations for node {node1}:")

for node, score in similar_nodes:

    print(f"Node {node}: {score}")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.6 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.1
    Uninstalling networkx-3.1:
      Successfully uninstalled networkx-3.1


Computing transition probabilities:   0%|          | 0/5 [00:00<?, ?it/s]

Similarity score between node 1 and node 5: 0.9281479716300964
Top 3 recommendations for node 1:
Node 2: 0.9881318211555481
Node 3: 0.9700296521186829
Node 4: 0.9543817639350891
